# Day Trader Dashboard

In [1]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
from newsapi import NewsApiClient 
import datetime

In [18]:
# Define the list of stock symbols you want to include
stock_symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', '^GSPC']
stock_names = {
    'AAPL': 'Apple Inc.',
    'MSFT': 'Microsoft Corporation',
    'GOOGL': 'Alphabet Inc.',
    'AMZN': 'Amazon.com, Inc.',
    'TSLA': 'Tesla, Inc.',
    'NVDA': 'NVIDIA Corporation',
    '^GSPC': 'S&P 500 Index'
}

# Fetch stock data using yfinance
def fetch_stock_data(symbols):
    data = {}
    end_date = datetime.date.today().strftime('%Y-%m-%d')
    for symbol in symbols:
        stock_data = yf.download(symbol, start='2020-01-01', end=end_date)
        stock_data['Symbol'] = symbol
        data[symbol] = stock_data
    return pd.concat(data.values())

# Load stock data
df = fetch_stock_data(stock_symbols)
df.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [19]:
# Specify the file path
file_path = '/Users/oliverdreger/Documents/MyAPIs/api_key_newsapi.txt'

# Read the API key from the file
try:
    with open(file_path, 'r') as file:
        # Read the first line of the file (assuming the API key is on the first line)
        api_key = file.readline().strip()  # Use strip() to remove any leading/trailing whitespace

    # Print the API key (for debugging purposes; remove or comment out in production)
    print(f"API Key: api_key loaded")

except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

# Initialize the News API client
api = NewsApiClient(api_key=api_key)

API Key: api_key loaded


In [20]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1('Day Trader Dashboard'),
    dcc.Dropdown(
        id='stock-symbol',
        options=[{'label': symbol, 'value': symbol} for symbol in stock_symbols],
        value=stock_symbols[0]
    ),
    html.H2(id='stock-name'),
    dcc.DatePickerRange(
        id='date-picker',
        start_date=df['Date'].min(),
        end_date=df['Date'].max(),
        display_format='YYYY-MM-DD'
    ),
    dcc.Graph(id='stock-graph'),
    html.H2('Recent News'),
    html.Div(id='news-list')
])

# Define the callback to update the graph, news, and stock name
@app.callback(
    [Output('stock-graph', 'figure'),
     Output('news-list', 'children'),
     Output('stock-name', 'children')],
    [Input('stock-symbol', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_graph_and_news(selected_symbol, start_date, end_date):
    # Update the stock graph
    filtered_df = df[(df['Symbol'] == selected_symbol) & 
                     (df['Date'] >= start_date) & 
                     (df['Date'] <= end_date)]
    
    fig = go.Figure(data=[go.Candlestick(
        x=filtered_df['Date'],
        open=filtered_df['Open'],
        high=filtered_df['High'],
        low=filtered_df['Low'],
        close=filtered_df['Close'],
        name=selected_symbol
    )])
    
    fig.update_layout(
        title=f'Stock Price Data for {selected_symbol}',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False
    )
    
    # Fetch recent news articles
    query = "S&P 500" if selected_symbol == "^GSPC" else selected_symbol
    news = api.get_everything(q=query, language='en', sort_by='publishedAt', page_size=5)
    news_items = []
    for article in news['articles']:
        news_items.append(html.Div([
            html.H3(html.A(article['title'], href=article['url'], target='_blank')),
            html.P(article['description']),
            html.P(f"Source: {article['source']['name']}"),
            html.P(f"Published at: {article['publishedAt']}")
        ]))
    
    # Get the descriptive name for the selected stock
    stock_name = stock_names.get(selected_symbol, "Unknown Stock")
    
    return fig, news_items, stock_name

# Run the app
app.run(debug=True)